In [1]:
# Instruções basiconas de sempre.

# Lidar com dataframes.
import numpy as np
import pandas as pd

# Lidar com gráficos.
import matplotlib.pyplot as plt
import seaborn as sns
# import plotly
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from plotting import (multiple_histograms_plot,
                      bar_plot_with_categorical,
                      plot_confusion_matrix,
                      plot_confusion_matrix_2,
                      plot_roc)

# Lidar com preparação de dados.
from data_prep import data_prep as dp # Eu que fiz esse modulinho ("uuuuuuuuuma bosts!").
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import (StandardScaler
                                   , MinMaxScaler)
from sklearn.model_selection import (train_test_split
                                     , cross_val_score
                                     , StratifiedKFold)

# Lidar com validação de modelos.
from sklearn.metrics import (confusion_matrix
                             , accuracy_score
                             , classification_report)

pd.set_option('display.max_columns', None)  
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 500)

!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


# Aquisitando os dados através de requisições a API do Brasil.IO e transformando JSON em um DataFrame
Seguindo as instruções em uma questão do [Stackoverflow](https://stackoverflow.com/questions/21104592/json-to-pandas-dataframe), temos:

In [2]:
#Bibliotecas necessárias para aquisitar as informações em json em requisitando uma API Rest.

import json
import requests

In [10]:
# Requisições por número da página

df_gastos_parlamentares = pd.DataFrame(list())

for numero in range (1,15):
    r = requests.get("https://brasil.io/api/dataset/gastos-deputados/cota_parlamentar/data?page="+str(numero))
    if r.status_code == 200:
        json_gastos_parlamentares = json.loads(r.content)
        regex_equal_symbol = json_gastos_parlamentares["next"].rfind("=")
#         print(json_gastos_parlamentares["next"][regex_equal_symbol+1:])
#         print(json_gastos_parlamentares["results"][0]["numano"])
        df_gastos_parlamentares = pd.concat([df_gastos_parlamentares
                                             , pd.io.json.json_normalize(json_gastos_parlamentares["results"])]
                                            , axis=0
                                            , join="outer"
                                           )
    
print(df_gastos_parlamentares.shape)

df_gastos_parlamentares.sample(5).T

(14000, 29)


,332,72,782,2,230
codlegislatura,55,56,55,55,55
datemissao,None,None,None,None,None
idecadastro,74060,74366,74352,160571,73701
idedocumento,0,0,0,0,0
indtipodocumento,0,0,0,0,0
nucarteiraparlamentar,188,418,31,473,291
nudeputadoid,1562,1671,1282,2315,188
nulegislatura,2015,2019,2015,2015,2015
numano,2018,2019,2018,2018,2018
numespecificacaosubcota,0,0,0,0,0


In [ ]:
# Requisições por ano. AVALIAR COMO QUERO USAR AS REQUISIÇÕES.


# ano_escolhido = 2020
# is_ano_escolhido = True
# pagina = 1

# while is_ano_escolhido:
#     r = requests.get("https://brasil.io/api/dataset/gastos-deputados/cota_parlamentar/data?page="+str(pagina))
#     # requests.get('https://brasil.io/api/dataset/gastos-deputados/cota_parlamentar/data?format=json')
#     if r.status_code == 200:
#         json_gastos_parlamentares = json.loads(r.content)
#         pagina += 1
#         if json_gastos_parlamentares["results"][0]["numano"] != ano_escolhido:
#             is_ano_escolhido = False
#         print(json_gastos_parlamentares["results"][0]["numano"])

# Transformando JSON em um DataFrame
Seguindo as instruções em uma questão do [Stackoverflow](https://stackoverflow.com/questions/21104592/json-to-pandas-dataframe), temos:

In [ ]:
df_gastos_parlamentares = pd.io.json.json_normalize(json_gastos_parlamentares["results"])
print(df_gastos_parlamentares.shape)
df_gastos_parlamentares.head()

In [ ]:
df_gastos_parlamentares.info()

In [ ]:
dp.serieNulos(df_gastos_parlamentares)

In [ ]:
df_gastos_parlamentares.drop(["vlrrestituicao"
                              , "datemissao"
                              , "txttrecho"
                              , "txtpassageiro"
                              , "txtdescricaoespecificacao"], axis=1, inplace=True)

In [ ]:
dp.cardinalidade(df_gastos_parlamentares)

**Análise preliminar:**

1. Os atributos / fatores / variáveis independentes que possuem cardinalidade igual a um não oferecem nenhum ganho de informação. Atributos 
**Ação:** Remover esses atributos.

2. Os atributos vlrdocumento e vlrliquido estão como objects, mas são floats.
**Ação:** Converter esses atributos para float.

In [ ]:
# Removendo os atributos com cardinalidade <= 1:

lista_atributos_para_remover = list(dp.cardinalidade(df_gastos_parlamentares)[dp.cardinalidade(df_gastos_parlamentares)["Cardinalidade"] <= 1]["Atributo"].values)

df_gastos_parlamentares.drop(lista_atributos_para_remover, axis=1, inplace=True)

In [ ]:
df_gastos_parlamentares.vlrdocumento = pd.to_numeric(df_gastos_parlamentares.vlrdocumento)
df_gastos_parlamentares.vlrliquido = pd.to_numeric(df_gastos_parlamentares.vlrliquido)

df_gastos_parlamentares.info()

In [ ]:
dp.cardinalidade(df_gastos_parlamentares)

In [ ]:
df_gastos_parlamentares.describe().T

In [ ]:
df_gastos_parlamentares.groupby(["sguf", "sgpartido"], axis=0)["vlrliquido"].mean()